# SED2 Builder API demo

"Things You Should Be Able To Do" were provided here: https://docs.google.com/document/d/1jZkaNhM_cOqMWtd4sJZ9b0VGXPTLsDKsRNI5Yvu4nOA/edit

In [1]:
from bigraph_viz import plot_bigraph, plot_flow, pf
from bigraph_viz.dict_utils import schema_keys
from demo_processes import process_registry  # this triggers the demo processes to register. TODO -- make this better

from sed2.builder import SEDBuilder

MODEL_PATH = 'demo_processes/Caravagna2010.xml'

In [2]:
import os
current_directory = os.getcwd()
print(f"The current working directory is: {current_directory}")

# Define the path to which you want to change the directory
new_directory = '/Users/eranagmon/code/sed2'
if not os.path.exists(new_directory):
    os.makedirs(new_directory)
os.chdir(new_directory)
print(f"Changed directory to: {new_directory}")


The current working directory is: /Users/eranagmon/code/sed2/notebooks
Changed directory to: /Users/eranagmon/code/sed2


## 1. Run a simulation from time start to time end with a given number of points/steps.  The run will return a 2D array of results.

In [3]:
# Initialize the SEDBuilder
sed = SEDBuilder()

# Load a model
sed.add_model(
    model_id='model1',
    source=MODEL_PATH,
    language='sbml',
)

# Set up the simulator
sed.add_simulator(
    simulator_id='simulator1',
    name='copasi',
    version='',
    kisao_id='',
)

# Begin the simulation experiment step
sed.start_step(step_id='run_simulation')

# Add a simulation run configuration
sed.add_simulation(
    simulation_id='run1',
    simulator_id='simulator1',
    model_id='model1',
    start_time=0,
    end_time=100,
    number_of_points=1000,
    observables=['observable1', 'observable2'],
)

# Execute the simulation
results = sed.execute()

# Assuming the 'execute' method returns the results directly.
# These results would be the 2D array with each row representing a time point
# and each column representing an observable's value at that time point.

# If you want to print or otherwise process the results, you can do so here.
# For example, you could print the results to the console or write them to a file.
print(results)


sed.save_archive()


None


## 2. Run a single steady-state simulation, The run returns a 1D array containing the steady state values.

In [4]:
# Initialize
sed = SEDBuilder()

# Load a model
sed.add_model(
    model_id='model1',
    source=MODEL_PATH,
    language='sbml',
)

# Set up the simulator
sed.add_simulator(
    simulator_id='simulator1',
    name='copasi',
    version='',
    kisao_id='0000407',  # steady-state method
)

# Begin the simulation step
sed.start_step(step_id='run_steady_state')

# Add a steady-state simulation run configuration
# No need for start_time, end_time, or number_of_points since it's a steady-state simulation
sed.add_simulation(
    simulation_id='steady_state_run',
    simulator_id='simulator1',
    model_id='model1',
)

# Execute the simulation
steady_state_values = sed.execute()

# Assuming the 'execute' method returns the results directly.
# The results would be the 1D array with each element representing the steady-state value of an observable.

# Process the results as needed, e.g., print to the console.
print(steady_state_values)


None


## 3. Set parameters and/or initial conditions and run time course or steady state evaluation.

In [5]:
# set run type to either 'time_course' or 'steady_state'
run_type = 'steady_state'

# Initialize
sed = SEDBuilder()

# Load a model
sed.add_model(
    model_id='model1',
    source=MODEL_PATH,
    language='sbml',
)

# load the model with changes, parameters or initial conditions
sed.add_model(
    model_id='model2',
    source=MODEL_PATH,
    language='sbml',
    changes={
        'param1': 10,
        'param2': 1,
    }
)

# Set up the simulator for time course or steady state
sed.add_simulator(
    simulator_id='simulator1',
    name='copasi',
    version='YourSimulatorVersion',
    kisao_id='0000407' if 'steady_state' else '',  # TODO -- get uniform time coarse id.
)

# Begin the simulation step
sed.start_step(step_id='parameter_tuning')

# Add a simulation run configuration for time course or steady state
if run_type == 'time_course':
    sed.add_simulation(
        simulation_id='sim_run',
        simulator_id='simulator1',
        model_id='model1',
        start_time=0,
        end_time=100,
        number_of_points=1000,
        algorithm_parameters={'type': 'time_course'}
    )
elif run_type == 'steady_state':
    sed.add_simulation(
        simulation_id='sim_run',
        simulator_id='simulator1',
        model_id='model1',
    )

# Execute the simulation
results = sed.execute()

# Process the results as needed, e.g., print to the console or save to a file.
print(results)


None


## 4. Repeat simulations any number of times and with any degree of nesting.  Any changes may be applied to parameters, initial conditions etc within the repeated simulations.  The results of the simulation will be collected into arrays.

## 5. Carry out an n-D parameter scan and return the results in an array.

In [6]:
# Initialize
sed = SEDBuilder()

# Define the parameters and their ranges for the scan
parameter_ranges = {
    'param1': [0.1, 1.0, 10.0],
    'param2': [20, 50, 100],
}

# Create models for each set of parameter values
for value1 in parameter_ranges['param1']:
    for value2 in parameter_ranges['param2']:
        model_id = f'model_param1_{value1}_param2_{value2}'
        sed.add_model(
            model_id=model_id,
            source=MODEL_PATH,
            language='sbml',
            changes=[{'target': 'param1', 'newValue': value1},
                     {'target': 'param2', 'newValue': value2}],
        )

# Set up the simulator
sed.add_simulator(
    simulator_id='simulator1',
    name='copasi',
    version='x.x',
    kisao_id='0000000',  # TODO get the KiSAO ID
)

# Begin a new step for the parameter scan
sed.start_step(step_id='parameter_scan')

# Add simulations for each model
for value1 in parameter_ranges['param1']:
    for value2 in parameter_ranges['param2']:
        model_id = f'model_param1_{value1}_param2_{value2}'
        sed.add_simulation(
            simulation_id=f'simulation_{model_id}',
            simulator_id='simulator1',
            model_id=model_id,
            start_time=0,
            end_time=100,
            number_of_points=1000,
            observables=['observable1', 'observable2'],  # Specify observables
        )

# Execute all the steps in the simulation experiment design
results = sed.execute()

## 6. Load experimental data and compare it with simulation data.

## 19. Run multiple stochastic simulations, compute means and standard deviations.

In [7]:
# Initialize
sed = SEDBuilder()

# Load an SBML model
sed.add_model(
    model_id='model1',
    source=MODEL_PATH,
    language='sbml',
)

# Set up copasi with the Gillespie algorithm
sed.add_simulator(
    simulator_id='gillespie_simulator',
    name='copasi',
    version='',
    kisao_id='0000029',  # Gillespie algorithm
)

# Begin the first step - running stochastic simulations
sed.start_step(step_id='stochastic_simulations')

# Define the number of simulations
n_simulations = 100
# Run the defined number of stochastic simulations
for i in range(n_simulations):
    sed.add_simulation(
        simulation_id=f'simulation_{i}',
        simulator_id='gillespie_simulator',
        model_id='model1',
        start_time=0,
        end_time=100,
        number_of_points=1000,
        # Assuming that 'observables' are the species or parameters of interest
        observables=['species1', 'species2'],
    )

# Begin the second step - data processing for mean and standard deviation
sed.start_step(step_id='data_processing')

# Add data generator for mean and standard deviation for the observables
# We are assuming that the SEDBuilder will internally handle the retrieval of
# simulation results and compute the required statistics
for observable in ['species1', 'species2']:
    sed.add_data_generator(
        data_id=f'stats_{observable}',
        observables=observable,
        operations=['mean', 'standard_deviation']
    )

# Begin the third step - plotting the results
sed.start_step(step_id='plotting')

# Plot the mean and standard deviation for each observable
for observable in ['species1', 'species2']:
    sed.add_visualization(
        plot_id=f'plot_stats_{observable}',
        plot_type='line',
        title=f'Statistics of {observable} Concentration',
        x_label='Time',
        y_label='Concentration',
        legend=['Mean', 'Standard Deviation']
    )

# Execute the simulation experiment
sed.execute()

## 21. Run a simulation, change the structure of the model, rerun the simulation, compare.

In [12]:
# Initialize the SEDBuilder
sed = SEDBuilder()

# Load an SBML model
sed.add_model(
    model_id='model1',
    source=MODEL_PATH,
    language='sbml',
)

# Set up the simulator
sed.add_simulator(
    simulator_id='simulator1',
    name='copasi',
    version='',
    kisao_id='',
)

# Begin the first step - run the initial simulation
sed.start_step(step_id='initial_simulation')

# Add the first simulation
sed.add_simulation(
    simulation_id='initial_run',
    simulator_id='simulator1',
    model_id='model1',
    start_time=0,
    end_time=100,
    number_of_points=1000,
    observables=['observable1', 'observable2'],
)

# Save the initial results for comparison
sed.add_data_generator(
    data_id='initial_results',
    observables=['observable1', 'observable2'],
    operations=['value']
)

# Begin the second step - modify the structure of the model
sed.start_step(step_id='modify_model')

# Perform modifications on the model, e.g., change parameter values, add or remove reactions
sed.add_model(
    model_id='model2',
    source=MODEL_PATH,
    language='sbml',
    changes=[
        {'type': 'parameter', 'id': 'param1', 'new_value': 1.5},
        # Add more changes as needed
    ]
)

# Begin the third step - run the simulation with the modified model
sed.start_step(step_id='modified_simulation')

# Add the second simulation using the modified model
sed.add_simulation(
    simulation_id='modified_run',
    simulator_id='simulator1',
    model_id='model1',
    start_time=0,
    end_time=100,
    number_of_points=1000,
    observables=['observable1', 'observable2'],
)

# Save the modified results for comparison
sed.add_data_generator(
    data_id='modified_results',
    observables=['observable1', 'observable2'],
    operations=['value']
)

# Begin the fourth step - compare the results
sed.start_step(step_id='compare_results')

# Compare the results of the initial and modified simulations
sed.add_data_generator(
    data_id='compare_results',
    # data_id1='initial_results',
    # data_id2='modified_results',
    generator_id='difference',  # Assuming 'difference' is a supported comparison type
)

# Execute the simulation experiment
sed.execute()


Exception: process "YourSimulatorName" not found